# Loading models from "[Why Do Better Loss Functions Lead to Less Transferable Features?](https://arxiv.org/abs/2010.16402)"

Copyright 2022 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import tensorflow as tf
import numpy as np

!git clone https://github.com/google-research/google-research.git google_research --depth=1
from google_research.loss_functions_transfer import resnet_preprocessing
from google_research.loss_functions_transfer import load_model



# Loading and running the SavedModels.


See LOSS_HYPERPARAMETERS in [load_model.py](https://github.com/google-research/google-research/blob/master/loss_functions_transfer/load_model.py)
for valid choices of `loss_name`. There are 8 seeds per loss function, so seed; seed should be in `{0, ..., 7}`.

In [ ]:
import tensorflow_datasets as tfds
from tqdm import tqdm

loss_name = 'softmax'
seed = 0

# Load SavedModel.
model = tf.saved_model.load(
    f'{load_model.BASE_PATH}/savedmodels/{loss_name}/seed{seed}')
model = model.signatures['serving_default']

# Construct dataset.
dataset = tfds.load('imagenet_v2', split='test')
# For this Colab, we use ImageNet-V2 for eval because it can be downloaded w/o
# manual intervention. To use ImageNet instead, uncomment below.
# dataset = tfds.load('imagenet2012', split='validation')

 # Set up preprocessing.
def preprocess_fn(example):
  return {
      'image': resnet_preprocessing.preprocess_image(
          example['image'], 224, 224, is_training=False),
      'label': example['label']
  }
dataset = dataset.map(preprocess_fn).batch(128, drop_remainder=False)

# Perform evaluation.
correct = 0
n = 0
for batch in tqdm(dataset):
  # The model returns the output of each block as well as the average pooling,
  # layer, but we use only the final outputs here.
  outputs = model(image=batch['image'])['outputs']
  # The model outputs 1001 classes, but ImageNet contains only 1000. The first
  # class is an unused background class. We drop it when evaluating.
  outputs = outputs[:, 1:]
  predictions = tf.argmax(outputs, -1)
  correct += tf.reduce_sum(tf.cast(predictions == batch['label'], tf.int64))
  n += batch['label'].shape[0]
print()
print(f'Accuracy: {correct / n * 100:.1f}%')

# Constructing, restoring, and running the checkpoint in graph mode.

In [ ]:
import numpy as np

loss_name = 'logit_penalty'
seed = 0

# Images should be preprocessed using:
# resnet_preprocessing.preprocess_image(image, 224, 224, is_training=False)
# (See cell above for an example using tensorflow_datasets.)
inputs_np = np.zeros((1, 224, 224, 3))
labels_np = np.pad([[1]], ((0, 0), (1000, 0)))

with tf.Graph().as_default():
  inputs = tf.compat.v1.placeholder(tf.float32, (None, 224, 224, 3))
  labels = tf.compat.v1.placeholder(tf.float32, (None, 1001))
  loss, endpoints = load_model.build_model_and_compute_loss(
      loss_name=loss_name, inputs=inputs, labels=labels,
      is_training=False)
  with tf.compat.v1.Session() as sess:
    load_model.restore_checkpoint(loss_name, seed, sess)
    loss_np, endpoints_np = sess.run((loss, endpoints),
             feed_dict={inputs: inputs_np, labels: labels_np})

{k: v.shape for k, v in endpoints_np.items()}